In [ ]:
import pandas as pd

In [ ]:
import tensorflow as tf
num_gpus_available = len(tf.config.experimental.list_physical_devices('GPU'))
print("Num GPUs Available: ", num_gpus_available)
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv("complete_preprocessing.csv",encoding = "utf-8",sep = ',')

In [ ]:
df_dev = pd.read_csv("dev_complete_preprocessing.csv",encoding = "utf-8",sep = ',')

In [ ]:
df_test = pd.read_csv("test_complete_preprocessing_stp_frsc.csv",encoding = "utf-8",sep = ',')

In [ ]:
df['note'] = df['note'].astype('int')
df_dev['note'] = df_dev['note'].astype('int')

In [ ]:
df

In [ ]:
df_dev

In [ ]:
df_test

In [ ]:
df = df.reset_index()
df_dev = df_dev.reset_index()

In [ ]:
df.drop(columns=['index'],inplace=True)
df_dev.drop(columns=['index'],inplace=True)

In [ ]:
from keras.preprocessing.text import text_to_word_sequence
def maxNumWords360(text):
    l=text_to_word_sequence(str(text))
    if len(l) >= 360:
        l = l[:360]
    return " ".join(l)

maxWordsNum = 0
def maxNumWords(text):
    global maxWordsNum
    l = text_to_word_sequence(str(text))
    if len(l) > maxWordsNum:
        maxWordsNum = len(l)

In [ ]:
df.commentaire.apply(maxNumWords)
print("Max num of words per sentence in the training set is: ",maxWordsNum)
maxWordsNum = 0
df_dev.commentaire.apply(maxNumWords)
print("Max num of words per sentence in the dev set is: ",maxWordsNum)
maxWordsNum = 0

df.commentaire=df.commentaire.apply(maxNumWords360)
df_dev.commentaire=df_dev.commentaire.apply(maxNumWords360)

df.commentaire.apply(maxNumWords)
print("Max num of words per sentence in the training set is: ",maxWordsNum)
maxWordsNum = 0
df_dev.commentaire.apply(maxNumWords)
print("Max num of words per sentence in the dev set is: ",maxWordsNum)
maxWordsNum = 0

In [ ]:
df_test.commentaire.apply(maxNumWords)
print("Max num of words per sentence in the training set is: ",maxWordsNum)
maxWordsNum = 0

df_test.commentaire=df_test.commentaire.apply(maxNumWords360)

df_test.commentaire.apply(maxNumWords)
print("Max num of words per sentence in the training set is: ",maxWordsNum)

In [ ]:
import os
import re
import nltk
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from sklearn.datasets import fetch_20newsgroups
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from gensim.test.utils import common_texts
from gensim.models import Word2Vec
from keras.layers import Dropout,BatchNormalization
from keras import regularizers
import gensim
from gensim.models import FastText
import nltk
from nltk.tokenize import SpaceTokenizer

In [ ]:
X_train = df.commentaire.values
y_train = df.note.values
X_val = df_dev.commentaire.values
y_val = df_dev.note.values

In [ ]:
X_test = df_test.commentaire.values

In [ ]:
tokenizer = Tokenizer(num_words=100000)
tokenizer.fit_on_texts(X_train)

In [ ]:
vocab_size = len(tokenizer.index_word) + 1
print('Vocab Size:', vocab_size)

In [ ]:
X_train_token = tokenizer.texts_to_sequences(X_train)
X_val_token = tokenizer.texts_to_sequences(X_val)

In [ ]:
X_test_token = tokenizer.texts_to_sequences(X_test)

In [ ]:
sequence_len = 360
X_train_token = pad_sequences(X_train_token, padding='post', maxlen=sequence_len)
X_val_token = pad_sequences(X_val_token, padding='post', maxlen=sequence_len)

In [ ]:
X_test_token = pad_sequences(X_test_token, padding='post', maxlen=sequence_len)

# Word2vec Model + Embedding Matrix


In [ ]:
train_sents = []
for i in range(len(X_train)):
  train_sents_intr = SpaceTokenizer().tokenize(X_train[i])
  train_sents.append(train_sents_intr)

In [ ]:
model_w2v_xtrain = Word2Vec(sentences=train_sents, size=100, sg=1, hs=1)

In [ ]:
embedding_matrix_word2vec = np.zeros((vocab_size, 100))
for index,word in enumerate(tokenizer.word_index):
  try :
    #model_w2v.wv[word]
    idx = tokenizer.word_index.get(word)
    embedding_matrix_word2vec[idx] = np.array(model_w2v.wv[word], dtype=np.float32)[:100]
  except:
    continue

# Fast Text + Embedding Matrix


In [ ]:
train_sents = []
for i in range(len(X_train)):
  train_sents_intr = SpaceTokenizer().tokenize(X_train[i])
  train_sents.append(train_sents_intr)

In [ ]:
model_fstxt = FastText(train_sents, size=100, window=5, min_count=5, workers=4,sg=1)

In [ ]:
embedding_matrix_fast_text = np.zeros((vocab_size, 100))
for index,word in enumerate(tokenizer.word_index):
  try :
    idx = tokenizer.word_index.get(word)
    embedding_matrix_fast_text[idx] = np.array(model_fstxt.wv[word], dtype=np.float32)[:100]
  except:
    continue

# CNN with keras tokenizer


In [ ]:
embedding_dim = 100
droprate=0.25
model = Sequential()
model.add(layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=sequence_len))
model.add(layers.Conv1D(filters=128, kernel_size=5, activation='relu',kernel_regularizer=regularizers.l2(l=0.01)))
model.add(BatchNormalization())
model.add(layers.GlobalMaxPool1D())
model.add(Dropout(droprate))
model.add(layers.Dense(64, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(droprate))
model.add(layers.Dense(10, activation='softmax'))
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
history = model.fit(X_train_token, y_train, epochs=7, validation_data=(X_val_token, y_val), batch_size=128)

# CNN Word2Vec + Fast Text

In [ ]:
embedding_dim = 100
droprate=0.25
model = Sequential()
# Modifier le paramètre weights pour changer de word2vec à fasttext ou inversement
model.add(layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=sequence_len, weights = [embedding_matrix_fast_text], trainable = True))
model.add(layers.Conv1D(filters=128, kernel_size=5, activation='relu',kernel_regularizer=regularizers.l2(l=0.01)))
model.add(BatchNormalization())
model.add(layers.GlobalMaxPool1D())
model.add(Dropout(droprate))
model.add(layers.Dense(64, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(droprate))
model.add(layers.Dense(10, activation='softmax'))
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
history = model.fit(X_train_token, y_train, epochs=7, validation_data=(X_val_token, y_val), batch_size=128)